- Import core and 3rd party modules
- Move current working directory to project root (if needed) and import project modules
- Set data paths and other constants

In [ ]:
######################################
# importing core and 3rd-party modules
######################################

import os
import pickle
import re
import scipy
from pathlib import Path
from winsound import Beep
from contextlib import suppress

import matplotlib as mpl

mpl.use("nbAgg")
import numpy as np
from IPython.core.debugger import set_trace
from matplotlib import pyplot as plt
from scipy.fft import fft, ifft

###############################################
# Move to project root to easily import modules
###############################################

try: # avoid changes if already set
    print("Working from: ", PROJECT_ROOT)
except NameError:
    try:  # running from Spyder
        PROJECT_ROOT = Path(__file__).resolve()
    except NameError:  # running as Jupyter Notebook
        PROJECT_ROOT = Path(os.getcwd()).resolve().parent.parent
    os.chdir(PROJECT_ROOT)
    print("Working from: ", PROJECT_ROOT)

from data_analysis.correlation_function import (CorrFunc, SFCSExperiment,
                                                SolutionSFCSMeasurement)
from data_analysis.software_correlator import (CorrelatorType,
                                               SoftwareCorrelator)
from utilities.display import Plotter, get_gradient_colormap
from utilities.file_utilities import (default_system_info, load_mat,
                                      load_object, save_object,
                                      save_processed_solution_meas)
from utilities.helper import Limits, fourier_transform_1d

#################################################
# Setting up data path and other global constants
#################################################

DATA_ROOT = Path("D:\OneDrive - post.bgu.ac.il\gSTED_sFCS_Data")  # Laptop/Lab PC (same path)
# DATA_ROOT = Path("D:\???")  # Oleg's
DATA_TYPE = "solution"

FORCE_PROCESSING = False
FORCE_PROCESSING = True

## Comparing afterpulsing subtration: whitenoise autocorrelation (Halogen lamp calibration) vs. cross-correlating gates

Here I attempt to compare the afterpulsing gotten from auto-correlating white noise, to this gotten from cross-correlating valid photons from "white-noise" photons in fluorescent sample measurements.

First, let's quickly get the default system afterpulsing currently used (matching the currently used detector settings).
For this we can load some measurement to get some lag:

In [ ]:
# DATA_DATE = "29_03_2022"; confocal_template = "bp300_20uW_angular_exc_172325_*.pkl"; label = "Free-Running 300 bp"
# DATA_DATE = "29_03_2022"; confocal_template = "bp300_20uW_200mW_angular_sted_174126_*.pkl"; label = "Free-Running 300 bp STED"
# DATA_DATE = "30_01_2022"; confocal_template = "yoyo300bp500nW_angular_sted_125650_*.pkl"; label = "Old Detector Free-Running 300 bp STED"

# DATA_DATE = "10_05_2018"; confocal_template = "bp300_angular_exc_*.mat"; label = "MATLAB Free-Running 300 bp"
DATA_DATE = "10_05_2018"
confocal_template = "bp300_angular_sted_*.mat"
label = "MATLAB Free-Running 300 bp STED"
# DATA_DATE = "20_08_2018"; confocal_template = "EdU300bp_angular_sted_*.mat"; label = "MATLAB Free-Running EdU 300 bp STED"

# DATA_DATE = "06_04_2022"; confocal_template = "atto_FR_angular_exc_141224_*.pkl"; label = "Free-Running ATTO"
# DATA_DATE = "13_03_2022"; confocal_template = "atto_12uW_FR_static_exc_182414_*.pkl"; label = "Free-Running static ATTO"

DATA_PATH = DATA_ROOT / DATA_DATE / DATA_TYPE

# load experiment
exp = SFCSExperiment(name=label)
exp.load_experiment(
    confocal_template=DATA_PATH / confocal_template,
    should_plot=True,
    should_use_preprocessed=not FORCE_PROCESSING,
    should_re_correlate=True,
)

# save processed data (to avoid re-processing)
exp.save_processed_measurements()

# Show countrate
print(f"Count-Rate: {exp.confocal.avg_cnt_rate_khz} kHz")

Now, using the same measurement, we'll try cross-correlating the 'fluorescence' and 'calibration' photons.

First, we need to calibrate the TDC, in order to know which photons belong in each group:

In [ ]:
exp.calibrate_tdc()

Next, let's cross-correlate the fluorescence and white-noise photons. This can be done by choosing gating once from 0 to 40 ns, then from 40 to 100 (or np.inf) ns:

In [ ]:
meas = exp.confocal
meas.xcf = {}  # "halogen_afterpulsing": meas.cf["confocal"].afterpulse}

gate1_ns = Limits(2, 10)
gate2_ns = Limits(35, 85)

corr_names = ("AB",)
CF_list = meas.cross_correlate_data(
    cf_name="fl_vs_wn",
    corr_names=corr_names,
    gate1_ns=gate1_ns,
    gate2_ns=gate2_ns,
    should_subtract_bg_corr=True,
    should_subtract_afterpulse=False,
)

CF_dict = {xx: CF_xx for xx, CF_xx in zip(corr_names, CF_list)}

for CF in CF_dict.values():
    CF.average_correlation()

# plotting all corrfuncs (from the experiment):
exp.plot_correlation_functions(y_field="avg_cf_cr", y_scale="log", ylim=Limits(5e1, 5e4))
exp.plot_correlation_functions(y_field="avg_cf_cr", ylim=Limits(5e1, 5e4))

Now let's try to use the cross-correlation as the afterpulsing:

In [ ]:
ap_factor = 1.05  # 1

# calculate afterpulsing from cross-correlation
countrate_a = np.mean([countrate_pair.a for countrate_pair in CF_dict["AB"].countrate_list])
countrate_b = np.mean([countrate_pair.b for countrate_pair in CF_dict["AB"].countrate_list])
inherent_afterpulsing = (
    countrate_b
    * (exp.confocal.gate_width_ns / gate2_ns.interval())
    * CF_dict["AB"].avg_cf_cr
    / countrate_a
)

# load experiment
exp_xcorr_as_ap = SFCSExperiment(
    name="Free-Running 300 bp, X-Correlation of Fluorescent vs. White-Noise as Afterpulsing"
)
exp_xcorr_as_ap.load_experiment(
    confocal_template=DATA_PATH / confocal_template,
    should_plot=False,
    should_plot_meas=False,
    should_use_preprocessed=True,
    should_re_correlate=True,  # Need to re-process with ext. afterpulsing
    external_afterpulsing=inherent_afterpulsing * ap_factor,
)

Let's look at them together:

In [ ]:
with Plotter(super_title=label, ylim=(-300, exp.confocal.cf["confocal"].g0 * 1.5)) as ax:
    exp.confocal.cf["confocal"].plot_correlation_function(
        parent_ax=ax, x_field="vt_um", plot_kwargs=dict(label="Regular")
    )
    exp_xcorr_as_ap.confocal.cf["confocal"].plot_correlation_function(
        parent_ax=ax, x_field="vt_um", plot_kwargs=dict(label="BA XCorr as Afterpulsing")
    )
    ax.legend()

with Plotter(
    super_title="Afterpulsing", x_scale="log", y_scale="linear", xlim=(1e-4, 1e0), ylim=(1e-2, 1e4)
) as ax:
    lag = exp.confocal.cf["confocal"].lag
    ax.plot(lag, exp.confocal.cf["confocal"].afterpulse, label="Halogen AutoCorr")
    ax.plot(CF_dict["AB"].lag, inherent_afterpulsing, label="X Corr")
    ax.legend()

## Afterpulsing removal through de-convolution

### Derivation
Let us define $F(t)$ as the signal at time $t$, and $p(t)$ as the probability density per unit time of afterpulsing at time $t$. We have, therefore:
$$
\delta I(t) = \int \delta F(t-t')p(t')dt'
$$

Presenting $F$ and $p$ as reverse Fourier transforms
$$
\delta F(t) = \frac{1}{\sqrt{2\pi}}\int d\omega\delta F(\omega)e^{i\omega t}
$$
$$
\delta p(t) = \frac{1}{\sqrt{2\pi}}\int d\omega'\delta p(\omega')e^{i\omega't}
$$

and plugging back in:
$$
\delta I(t) = \frac{1}{2\pi}\int d\omega d\omega'\delta F(\omega)p(\omega') \int_{-\infty}^\infty e^{i[\omega(t-t')+\omega't']}dt'
$$

we notice that the last integral yields a delta function $e^{i\omega t}\delta(\omega'-\omega)$:
$$
= \int\delta\omega F(\omega)p(\omega)e^{i\omega t}
$$

Now, for the ACF. We conjugate $\delta I(t')$ for convenience (it is real):
$$
G(t)=\langle\delta I(0)\delta I(t)\rangle = \frac{1}{2T}\int dt' \langle\delta I^*(t')\delta I(t'+t)\rangle
$$
$$
= \frac{1}{2T}\frac{1}{(2\pi)^2}\int d\omega d\omega'\delta F^*(\omega)p^*(\omega)\delta F(\omega')p(\omega')\int dt' e^{-i(\omega'-\omega) t'}e^{i\omega' t}
$$

and after noticing yet another delta function:
$$
=\frac{1}{2T}\frac{1}{(2\pi)^2}\int d\omega |F(\omega)|^2|p(\omega)|^2e^{i\omega t}
$$

Noticing that this is a reverse Fourier transform, we can perform a Fourier transform to extract the product of $F$ and $p$:
$$
FT\left[G(t)\right]=G(\omega)=\frac{1}{2T}\frac{1}{(2\pi)^{3/2}}|F(\omega)|^2|p(\omega)|^2
$$

So, the Fourier transform of the afterpulsed ACF is proportional to the Fourier transform of the afterpulsing-clean ACF times the Fourier transform of the afterpulsing ACF. In other words, the afterpulsing seperates from the signal in Fourier  space.

We already saw (see above, for old detector data) that cross-correlating signal at different gates (signal vs. white noise tail) yields the afterpulsing. Until now we subtracted it from the ACF on grounds that the afterpulsing was strong at lag-times where the clean ACF was unintersting. We can attempt now to get rid of afterpulsing in a more general way:
$$
\frac{G_{ap\ signal}(\omega)}{G_{ap}(\omega)} = \frac{|F(\omega)|^2|p(\omega)|^2}{|p(\omega)|^2}=|F(\omega)|^2 = G_{signal}(\omega)
$$

Where $G_{ap\ signal}(\omega)$ is Fourier-transformed afterpulsed signal, $G_{ap}(\omega)$ is the Fourier-transformed cross-correlated signal afterpulsing and $G_{signal}(\omega)$ is the afterpulse-free Fourier-transformed ACF.

We may need to reverse-transform $G_{signal}(\omega)$, since we need the regular-time ACF to perform the Hankel transform on (spatial 2D Fourier transform) for deconvoluting the PSF from the structure factor (in exactly the same process!).

NOTE: is there a way to do it without reverse-transforming?

### Testing

Here we would like to test the above method on old detector measurements (since for them we can compare this method to the well-tested subtraction method).

We begin by loading the measurement and calibrating the TDC:

In [ ]:
DATA_DATE = "10_05_2018"
confocal_template = "bp300_angular_sted_*.mat"
label = "MATLAB Free-Running 300 bp STED"

DATA_PATH = DATA_ROOT / DATA_DATE / DATA_TYPE

# load experiment
exp = SFCSExperiment(name=label)
exp.load_experiment(
    confocal_template=DATA_PATH / confocal_template,
    should_plot=True,
    should_use_preprocessed=True, # TODO: load anew if not found
    should_re_correlate=False, # True
    should_subtract_afterpulse=False,
)

# save processed data (to avoid re-processing)
exp.save_processed_measurements()

# Show countrate
print(f"Count-Rate: {exp.confocal.avg_cnt_rate_khz} kHz")

# calibrate TDC
exp.calibrate_tdc(should_plot=False)

Now, let's get the afterpulsing from cross-corralting gates:

In [ ]:
meas = exp.confocal
meas.xcf = {}  # "halogen_afterpulsing": meas.cf["confocal"].afterpulse}

gate1_ns = Limits(2, 10)
gate2_ns = Limits(35, 85)

corr_names = ("AB",)
XCF = meas.cross_correlate_data(
    cf_name="fl_vs_wn",
    corr_names=corr_names,
    gate1_ns=gate1_ns,
    gate2_ns=gate2_ns,
    should_subtract_bg_corr=True,
    should_subtract_afterpulse=False,
)[0]

XCF.average_correlation()

Now, we need to perform Fourier transforms. To prepare the functions for the transform we would do well to trim the noisy tail end and to symmetrize them.

Let's define them and take a look first:

In [ ]:
# defining the ACF of the afterpulsing (from xcorr) and the afterpulsed signal
G_ap_t = np.copy(XCF.avg_cf_cr)
G_ap_signal_t = np.copy(exp.confocal.cf["confocal"].avg_cf_cr)
lag = np.copy(XCF.lag)

# plotting
with Plotter(super_title="Logarithmic Scale",
    xlim=(1e-3, 1e1), ylim=(-500, exp.confocal.cf["confocal"].g0 * 1.3), x_scale="log"
) as ax:

    ax.plot(lag, G_ap_signal_t, label="Afterpulsed Signal")
    ax.plot(lag, G_ap_t, label="X-Corr Afterpulsing")
    ax.legend()
    
with Plotter(super_title="Linear Scale",
    xlim=(-1e-2, 1e-1), ylim=(-1e4, 1e5), x_scale="linear"
) as ax:

    ax.plot(lag, G_ap_signal_t, label="Afterpulsed Signal")
    ax.plot(lag, G_ap_t, label="X-Corr Afterpulsing")
    ax.legend()

Checking out the probability density at bin 0:

In [ ]:
lag[G_ap_t == max(G_ap_t)]

In [ ]:
n = 0
G_ap_signal_t[n]
G_ap_signal_t[n] * np.diff(lag)[n] * 1e-3

In [ ]:
G_ap_t[0] * np.diff(lag)[n] * 1e-3

Cutting of the tail at lag of 1 ms: (later should gaussian interpolate)

In [ ]:
G_ap_signal_t[lag > 1] = 0
G_ap_t[lag > 1] = 0

Testing our Fourier transform:

In [ ]:
# %debug
# define Gaussian on positive axis:
a = 1
sigma = 5
t = np.arange(int(1e4))
ft = a * np.exp(-(t/sigma)**2)

# show it
with Plotter(super_title="Gaussian",
    xlabel="$t$ (ms)",
    ylabel="f(t) (Normalized)",
    xlim=(-1e-2, sigma * 4),
) as ax:
    ax.plot(t, ft, 'o')

# Fourier transform it:
r, fr_interp, w, fw = fourier_transform_1d(
    t,
    ft,
    bin_size=1, # in ms (meaning 100 ns)
    should_symmetrize=False,
#     should_symmetrize=True,
#     should_make_q_symmetric=False,
    should_make_q_symmetric=True,
    lag_units_factor=1,
)

# show Fourier transform interpolation
with Plotter(super_title="Interpolation",
    xlim=(-sigma * 4, sigma * 4),
) as ax:
    ax.plot(t, ft, 'o', label="before interpolation")
    ax.plot(r, fr_interp, 'x', label="after interpolation")
    ax.legend()
    
# show vanilla FFT
fq = scipy.fft.fft(fr_interp)
with Plotter(super_title="Quick and dirty FFT",
) as ax:
    ax.plot(np.real(fq), label="real part")
    ax.plot(np.imag(fq), label="imaginary part")
    ax.legend()

# show our Fourier transform
with Plotter(super_title="Our Fourier Transform",
    xlabel="$\omega$ (UNITS?)",
    ylabel="f($\omega$) (UNITS?)",
#     xlim=(-1e-2, 1e-1), ylim=(-1e4, 1e5), x_scale="linear"
) as ax:
    ax.plot(w, np.real(fw), label="real part")
    ax.plot(w, np.imag(fw), label="imaginary part")
    ax.legend()

Fourier transform:

In [ ]:
# %debug
r, fr_interp, q, fq = fourier_transform_1d(
    lag,
    G_ap_signal_t,
    bin_size=1e-4, # in ms (meaning 100 ns)
    should_symmetrize=False,
#     should_symmetrize=True,
    should_make_q_symmetric=True,
#     should_gaussian_interpolate=True,
)

with Plotter(super_title="Interpolation",
    xlim=(-1e-3, 1e-3), x_scale="linear"
) as ax:
    ax.plot(lag, G_ap_signal_t * 1e-4 * 1e-3, 'o', label="before interpolation")
    ax.plot(r, fr_interp, 'x', label="after interpolation")
    ax.legend()

with Plotter(super_title="Fourier Transform",
    xlabel="$\omega$ (UNITS?)",
    ylabel="f($\omega$) (UNITS?)",
#     xlim=(-1e-2, 1e-1), ylim=(-1e4, 1e5), x_scale="linear"
) as ax:

    ax.plot(q, np.real(fq), label="real part")
    ax.plot(q, np.imag(fq), label="imaginary part")
    ax.legend()

Play sound when done:

In [ ]:
Beep(4000, 300)